<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -q
!wandb login

     |████████████████████████████████| 1.9 MB 10.3 MB/s 
     |████████████████████████████████| 182 kB 54.4 MB/s 
     |████████████████████████████████| 162 kB 52.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 162 kB 71.1 MB/s 
     |████████████████████████████████| 158 kB 49.2 MB/s 
     |████████████████████████████████| 157 kB 57.9 MB/s 
     |████████████████████████████████| 157 kB 7.0 MB/s 
     |████████████████████████████████| 157 kB 55.5 MB/s 
     |████████████████████████████████| 157 kB 56.1 MB/s 
     |████████████████████████████████| 157 kB 35.3 MB/s 
     |████████████████████████████████| 157 kB 52.3 MB/s 
     |████████████████████████████████| 157 kB 59.4 MB/s 
     |████████████████████████████████| 156 kB 59.0 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb

In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [3]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [4]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
vgg16 = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58900480/58889256 [==============================] - 0s 0us/step


In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbign and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [12]:
model_vgg16 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg16,
  layers.Flatten(),
  layers.Dense(1028, activation='relu'),
  layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
vgg16.trainable=False
model_vgg16.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 1028)              25791492  
                                                                 
 dropout (Dropout)           (None, 1028)              0         
                                                                 
 dense_3 (Dense)             (None, 5)                 5145      
                                                                 
Total params: 40,511,325
Trainable params: 25,796,637


In [13]:
model_vgg16.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [14]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "vgg16_2",
      tags = "without augment with dropout"
)

In [15]:
epochs=40
history = model_vgg16.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 4.8052 - accuracy: 0.5794

wandb: Adding directory to artifact (/content/wandb/run-20221007_072750-29xju7ug/files/model-best)... Done. 2.5s


19/19 [==============================] - 15s 759ms/step - loss: 4.8052 - accuracy: 0.5794 - val_loss: 0.5914 - val_accuracy: 0.8514
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.6442 - accuracy: 0.8378

wandb: Adding directory to artifact (/content/wandb/run-20221007_072750-29xju7ug/files/model-best)... Done. 2.4s


19/19 [==============================] - 14s 740ms/step - loss: 0.6442 - accuracy: 0.8378 - val_loss: 0.2235 - val_accuracy: 0.9257
Epoch 3/40
19/19 [==============================] - ETA: 0s - loss: 0.1754 - accuracy: 0.9493

...

19/19 [==============================] - 14s 723ms/step - loss: 0.0379 - accuracy: 0.9882 - val_loss: 0.0468 - val_accuracy: 0.9865
Epoch 7/40
19/19 [==============================] - 7s 337ms/step - loss: 0.0315 - accuracy: 0.9882 - val_loss: 0.0896 - val_accuracy: 0.9730
Epoch 8/40
19/19 [==============================] - 7s 340ms/step - loss: 0.0302 - accuracy: 0.9932 - val_loss: 0.0797 - val_accuracy: 0.9662
Epoch 9/40
19/19 [==============================] - 7s 339ms/step - loss: 0.0252 - accuracy: 0.9899 - val_loss: 0.0789 - val_accuracy: 0.9797
Epoch 10/40
19/19 [==============================] - 7s 343ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0600 - val_accuracy: 0.9797
Epoch 11/40
19/19 [==============================] - 7s 345ms/step - loss: 0.0118 - accuracy: 0.9983 - val_loss: 0.0548 - val_accuracy: 0.9865
Epoch 12/40
19/19 [==============================] - 7s 346ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0587 - val_accuracy: 0.9865
Epoch 13/40
1

wandb: Adding directory to artifact (/content/wandb/run-20221007_072750-29xju7ug/files/model-best)... Done. 2.3s


19/19 [==============================] - 14s 724ms/step - loss: 0.0075 - accuracy: 0.9983 - val_loss: 0.0444 - val_accuracy: 0.9865
Epoch 15/40
19/19 [==============================] - 7s 347ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0813 - val_accuracy: 0.9865
Epoch 16/40
19/19 [==============================] - ETA: 0s - loss: 0.0057 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221007_072750-29xju7ug/files/model-best)... Done. 2.2s


19/19 [==============================] - 14s 738ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.0381 - val_accuracy: 0.9865
Epoch 17/40
19/19 [==============================] - 7s 349ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0740 - val_accuracy: 0.9865
Epoch 18/40
19/19 [==============================] - 7s 349ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9865
Epoch 19/40
19/19 [==============================] - 7s 351ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0621 - val_accuracy: 0.9865
Epoch 20/40
19/19 [==============================] - 7s 355ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0467 - val_accuracy: 0.9865
Epoch 21/40
19/19 [==============================] - 7s 367ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0578 - val_accuracy: 0.9797
Epoch 22/40
19/19 [==============================] - 7s 355ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0618 - val_accuracy: 0.9865
Epoch 23/4

In [16]:
wandb.finish()

accuracy,▁▅▇█████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▇▇██▇▇███████████████▇██▇█████████████
val_loss,█▃▂▁▁▁▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,15
best_val_loss,0.03815
epoch,39
loss,0.00151
val_accuracy,0.98649
